In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
# conf.set('spark.ui.proxyBase'
# , '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4041')
conf.set('spark.sql.repl.eagerEval.enabled', True)
conf.set('spark.driver.memory','4g')
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.SQLContext.getOrCreate(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/28 13:13:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/28 13:13:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


/Users/poojaakki/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [61]:
# Read the csv files from datasets

#Fake news
df_fake = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/Fake.csv")
df_fake.show()

+--------------------+--------------------+-------+-----------------+
|               title|                text|subject|             date|
+--------------------+--------------------+-------+-----------------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|
| WATCH: Brand-New...|Just when you mig...|   News|December 21, 2017|
| Papa John’s Foun...|A centerpiece of ...|   News|December 21, 2017|
| WATCH: Paul Ryan..

In [62]:
#Truth news
df_true = spark.read.option("header", "true").csv("/Users/poojaakki/Desktop/projects/Fake-News-Detection-Data-Analysis/datasets/News _dataset/True.csv")
df_true.show()

+--------------------+--------------------+--------------------+------------------+
|               title|                text|             subject|              date|
+--------------------+--------------------+--------------------+------------------+
|As U.S. budget fi...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |
|U.S. military to ...|WASHINGTON (Reute...|        politicsNews|December 29, 2017 |
|Senior U.S. Repub...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |
|FBI Russia probe ...|WASHINGTON (Reute...|        politicsNews|December 30, 2017 |
|Trump wants Posta...|SEATTLE/WASHINGTO...|        politicsNews|December 29, 2017 |
|White House, Cong...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |
|Trump says Russia...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |
|Factbox: Trump on...|The following sta...|        politicsNews|December 29, 2017 |
|Trump on Twitter ...|The following sta...|        politicsNews|December 29,

In [63]:
# count of data and the. umber of coloumns
print((df_fake.count(), len(df_fake.columns)))
print((df_true.count(), len(df_true.columns)))

(23489, 4)
(21417, 4)


In [64]:
#Add the column for fake news and also a column for true news
from pyspark.sql.functions import lit

df_fake = df_fake.withColumn("Flag", lit("Fake_News"))
df_fake.show()

+--------------------+--------------------+-------+-----------------+---------+
|               title|                text|subject|             date|     Flag|
+--------------------+--------------------+-------+-----------------+---------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|Fake_News|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|Fake_News|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|Fake_News|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|Fake_News|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|Fake_News|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|Fake_News|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|Fake_News|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|Fake_News|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|Fake_News|
| WATCH: Brand-New...|Just when you mig.

In [65]:
df_true = df_true.withColumn("Flag", lit("True_News"))
df_true.show()

+--------------------+--------------------+--------------------+------------------+---------+
|               title|                text|             subject|              date|     Flag|
+--------------------+--------------------+--------------------+------------------+---------+
|As U.S. budget fi...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |True_News|
|U.S. military to ...|WASHINGTON (Reute...|        politicsNews|December 29, 2017 |True_News|
|Senior U.S. Repub...|WASHINGTON (Reute...|        politicsNews|December 31, 2017 |True_News|
|FBI Russia probe ...|WASHINGTON (Reute...|        politicsNews|December 30, 2017 |True_News|
|Trump wants Posta...|SEATTLE/WASHINGTO...|        politicsNews|December 29, 2017 |True_News|
|White House, Cong...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |True_News|
|Trump says Russia...|WEST PALM BEACH, ...|        politicsNews|December 29, 2017 |True_News|
|Factbox: Trump on...|The following sta...|        politicsN

In [66]:
#concat the two tables
df_concat = df_fake.unionByName(df_true, allowMissingColumns=True)
df_concat.show()

+--------------------+--------------------+-------+-----------------+---------+
|               title|                text|subject|             date|     Flag|
+--------------------+--------------------+-------+-----------------+---------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|Fake_News|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|Fake_News|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|Fake_News|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|Fake_News|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|Fake_News|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|Fake_News|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|Fake_News|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|Fake_News|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|Fake_News|
| WATCH: Brand-New...|Just when you mig.

In [67]:
#specifying the row number to each row

from pyspark.sql.functions import monotonically_increasing_id 

df_concat = df_concat.select("*").withColumn("id", monotonically_increasing_id())

df_concat.show()

+--------------------+--------------------+-------+-----------------+---------+---+
|               title|                text|subject|             date|     Flag| id|
+--------------------+--------------------+-------+-----------------+---------+---+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|Fake_News|  0|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|Fake_News|  1|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|Fake_News|  2|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|Fake_News|  3|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|Fake_News|  4|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|Fake_News|  5|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|Fake_News|  6|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|Fake_News|  7|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|Fake_Ne

In [68]:
print((df_concat.count(), len(df_concat.columns)))

(44906, 6)


In [69]:
#### has to randomly shuffle the data
import pyspark.sql.functions as F

df_concat = df_concat.select("*").orderBy(F.rand())
df_concat.show()

+--------------------+--------------------+------------+-------------------+---------+------------+
|               title|                text|     subject|               date|     Flag|          id|
+--------------------+--------------------+------------+-------------------+---------+------------+
|EPA chief wants s...|WASHINGTON (Reute...|politicsNews|     July 11, 2017 |True_News| 68719479551|
|Clinton outpaces ...|WASHINGTON/NEW YO...|politicsNews|     June 16, 2016 |True_News| 94489281463|
|DEMOCRATS ARE AFR...|Don t let Hillary...|    politics|       Apr 25, 2016|Fake_News| 34359739457|
|Danish minister r...|COPENHAGEN (Reute...|   worldnews|September 26, 2017 |True_News|120259086228|
|Brexit with no de...|LONDON (Reuters) ...|   worldnews|  November 7, 2017 |True_News|111669151362|
|Flint’s #CrookedM...|Flint s Democrat ...|   left-news|       Sep 14, 2016|Fake_News| 42949676178|
|COVER-UP? New Det...|Shawn Helton  21s...|     US_News| September 30, 2016|Fake_News| 51539609947|


In [70]:
# Remove the unwanted coloumn

df_concat =  df_concat.drop(df_concat["date"])
df_concat

title,text,subject,Flag,id
EPA chief wants s...,WASHINGTON (Reute...,politicsNews,True_News,68719479551
Clinton outpaces ...,WASHINGTON/NEW YO...,politicsNews,True_News,94489281463
DEMOCRATS ARE AFR...,Don t let Hillary...,politics,Fake_News,34359739457
Danish minister r...,COPENHAGEN (Reute...,worldnews,True_News,120259086228
Brexit with no de...,LONDON (Reuters) ...,worldnews,True_News,111669151362
Flint’s #CrookedM...,Flint s Democrat ...,left-news,Fake_News,42949676178
COVER-UP? New Det...,Shawn Helton 21s...,US_News,Fake_News,51539609947
Judge blocks prov...,"AUSTIN, Texas (Re...",politicsNews,True_News,68719478700
It Sure Looks Li...,The Republican Pa...,News,Fake_News,8589936837
Trump Comes For ...,Donald Trump seem...,News,Fake_News,592


In [71]:
# Remove the title for analysis

df_concat =  df_concat.drop(df_concat["title"])
df_concat

text,subject,Flag,id
WASHINGTON (Reute...,politicsNews,True_News,68719479551
WASHINGTON/NEW YO...,politicsNews,True_News,94489281463
Don t let Hillary...,politics,Fake_News,34359739457
COPENHAGEN (Reute...,worldnews,True_News,120259086228
LONDON (Reuters) ...,worldnews,True_News,111669151362
Flint s Democrat ...,left-news,Fake_News,42949676178
Shawn Helton 21s...,US_News,Fake_News,51539609947
"AUSTIN, Texas (Re...",politicsNews,True_News,68719478700
The Republican Pa...,News,Fake_News,8589936837
Donald Trump seem...,News,Fake_News,592


In [72]:
# In order to avaoid the confusion convert everything to lowercase
from pyspark.sql.functions import col, split, lower, regexp_replace, length

df_concat = df_concat\
.withColumn("text", regexp_replace(lower(col("text")), r"[^0-9a-z]", " "))\
.where(length("text")>0)

df_concat

text,subject,Flag,id
washington reute...,politicsNews,True_News,68719479551
washington new yo...,politicsNews,True_News,94489281463
don t let hillary...,politics,Fake_News,34359739457
copenhagen reute...,worldnews,True_News,120259086228
london reuters ...,worldnews,True_News,111669151362
flint s democrat ...,left-news,Fake_News,42949676178
shawn helton 21s...,US_News,Fake_News,51539609947
austin texas re...,politicsNews,True_News,68719478700
the republican pa...,News,Fake_News,8589936837
donald trump seem...,News,Fake_News,592


In [73]:
# remove stop words from it
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/poojaakki/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [79]:
df_concat = df_concat.withColumn('words',F.split(F.col('text'),' '))
df_concat

text,subject,Flag,id,words
washington reute...,politicsNews,True_News,68719479551,"[washington, , re..."
washington new yo...,politicsNews,True_News,94489281463,"[washington, new,..."
don t let hillary...,politics,Fake_News,34359739457,"[don, t, let, hil..."
copenhagen reute...,worldnews,True_News,120259086228,"[copenhagen, , re..."
london reuters ...,worldnews,True_News,111669151362,"[london, , reuter..."
flint s democrat ...,left-news,Fake_News,42949676178,"[flint, s, democr..."
shawn helton 21s...,US_News,Fake_News,51539609947,"[shawn, helton, ,..."
austin texas re...,politicsNews,True_News,68719478700,"[austin, , texas,..."
the republican pa...,News,Fake_News,8589936837,"[the, republican,..."
donald trump seem...,News,Fake_News,592,"[donald, trump, s..."


In [80]:
from pyspark.ml.feature import StopWordsRemover 

stopwordsRemovalFeature = StopWordsRemover(inputCol="words", 
                   outputCol="words without stop").setStopWords(stop)

In [81]:
from pyspark.ml import Pipeline

stopWordRemovalPipeline = Pipeline(stages=[stopwordsRemovalFeature])
pipelineFitRemoveStopWords = stopWordRemovalPipeline.fit(df_concat)

In [82]:
df_concat = pipelineFitRemoveStopWords.transform(df_concat)

In [83]:
df_concat

text,subject,Flag,id,words,words without stop
washington reute...,politicsNews,True_News,68719479551,"[washington, , re...","[washington, , re..."
washington new yo...,politicsNews,True_News,94489281463,"[washington, new,...","[washington, new,..."
don t let hillary...,politics,Fake_News,34359739457,"[don, t, let, hil...","[let, hillary, ch..."
copenhagen reute...,worldnews,True_News,120259086228,"[copenhagen, , re...","[copenhagen, , re..."
london reuters ...,worldnews,True_News,111669151362,"[london, , reuter...","[london, , reuter..."
flint s democrat ...,left-news,Fake_News,42949676178,"[flint, s, democr...","[flint, democrat,..."
shawn helton 21s...,US_News,Fake_News,51539609947,"[shawn, helton, ,...","[shawn, helton, ,..."
austin texas re...,politicsNews,True_News,68719478700,"[austin, , texas,...","[austin, , texas,..."
the republican pa...,News,Fake_News,8589936837,"[the, republican,...","[republican, part..."
donald trump seem...,News,Fake_News,592,"[donald, trump, s...","[donald, trump, s..."


In [89]:
# convert array of strings to strings
from pyspark.sql.functions import concat_ws

df_concat = df_concat.withColumn("text1", concat_ws(" ", df_concat["words without stop"]))
df_concat

text,subject,Flag,id,words,words without stop,text1
washington reute...,politicsNews,True_News,68719479551,"[washington, , re...","[washington, , re...",washington reute...
washington new yo...,politicsNews,True_News,94489281463,"[washington, new,...","[washington, new,...",washington new yo...
don t let hillary...,politics,Fake_News,34359739457,"[don, t, let, hil...","[let, hillary, ch...",let hillary cheer...
copenhagen reute...,worldnews,True_News,120259086228,"[copenhagen, , re...","[copenhagen, , re...",copenhagen reute...
london reuters ...,worldnews,True_News,111669151362,"[london, , reuter...","[london, , reuter...",london reuters ...
flint s democrat ...,left-news,Fake_News,42949676178,"[flint, s, democr...","[flint, democrat,...",flint democrat ma...
shawn helton 21s...,US_News,Fake_News,51539609947,"[shawn, helton, ,...","[shawn, helton, ,...",shawn helton 21s...
austin texas re...,politicsNews,True_News,68719478700,"[austin, , texas,...","[austin, , texas,...",austin texas re...
the republican pa...,News,Fake_News,8589936837,"[the, republican,...","[republican, part...",republican party ...
donald trump seem...,News,Fake_News,592,"[donald, trump, s...","[donald, trump, s...",donald trump seem...


In [91]:
df_concat = df_concat.drop("words","words without stop")

df_concat

text,subject,Flag,id,text1
washington reute...,politicsNews,True_News,68719479551,washington reute...
washington new yo...,politicsNews,True_News,94489281463,washington new yo...
don t let hillary...,politics,Fake_News,34359739457,let hillary cheer...
copenhagen reute...,worldnews,True_News,120259086228,copenhagen reute...
london reuters ...,worldnews,True_News,111669151362,london reuters ...
flint s democrat ...,left-news,Fake_News,42949676178,flint democrat ma...
shawn helton 21s...,US_News,Fake_News,51539609947,shawn helton 21s...
austin texas re...,politicsNews,True_News,68719478700,austin texas re...
the republican pa...,News,Fake_News,8589936837,republican party ...
donald trump seem...,News,Fake_News,592,donald trump seem...


In [92]:
df_concat = df_concat.drop("text")

df_concat

subject,Flag,id,text1
politicsNews,True_News,68719479551,washington reute...
politicsNews,True_News,94489281463,washington new yo...
politics,Fake_News,34359739457,let hillary cheer...
worldnews,True_News,120259086228,copenhagen reute...
worldnews,True_News,111669151362,london reuters ...
left-news,Fake_News,42949676178,flint democrat ma...
US_News,Fake_News,51539609947,shawn helton 21s...
politicsNews,True_News,68719478700,austin texas re...
News,Fake_News,8589936837,republican party ...
News,Fake_News,592,donald trump seem...


In [93]:
df_concat = df_concat.withColumnRenamed("text1","text")

df_concat

subject,Flag,id,text
politicsNews,True_News,68719479551,washington reute...
politicsNews,True_News,94489281463,washington new yo...
politics,Fake_News,34359739457,let hillary cheer...
worldnews,True_News,120259086228,copenhagen reute...
worldnews,True_News,111669151362,london reuters ...
left-news,Fake_News,42949676178,flint democrat ma...
US_News,Fake_News,51539609947,shawn helton 21s...
politicsNews,True_News,68719478700,austin texas re...
News,Fake_News,8589936837,republican party ...
News,Fake_News,592,donald trump seem...


In [113]:
# How many articles per subject?
# import matplotlib.pyplot as plt

print(df_concat.groupby('Flag').count())
#df_concat.groupby('subject','text').count().plot(kind="bar")
#plt.show()

+---------+-----+
|     Flag|count|
+---------+-----+
|Fake_News|23481|
|True_News|21417|
+---------+-----+

